In [ ]:
!pip install nni

In [ ]:
!pip install torchvision

In [ ]:
!pip install pytorch-lightning

# Neural Architecture Search

![blabla](https://nni.readthedocs.io/en/stable/_images/nas_abstract_illustration.png)

The process is similar to Hyperparameter Optimization, except that the target is the best architecture rather than hyperparameter. Concretely, an exploration strategy selects an architecture from a predefined search space. The architecture is passed to a performance evaluation to get a score, which represents how well this architecture performs on a particular task. This process is repeated until the search process is able to find the best architecture.

#### Example of a NN model


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import nni
from nni.nas.nn.pytorch import LayerChoice, ModelSpace, MutableDropout, MutableLinear

In [2]:
class Net(ModelSpace):  # should inherit ModelSpace rather than nn.Module
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(self.conv2(x), 2)
        x = torch.flatten(self.dropout1(x), 1)
        x = self.fc2(self.dropout2(F.relu(self.fc1(x))))
        output = F.log_softmax(x, dim=1)
        return output

### Model variations

In [3]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.depthwise = nn.Conv2d(in_ch, in_ch, kernel_size=3, groups=in_ch)
        self.pointwise = nn.Conv2d(in_ch, out_ch, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))

class MyModelSpace(ModelSpace):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)

        # LayerChoice is used to select a layer between Conv2d and DwConv.

        self.conv2 = LayerChoice([
            nn.Conv2d(32, 64, 3, 1),
            DepthwiseSeparableConv(32, 64)
        ], label='conv2')

        # nni.choice is used to select a dropout rate.
        # The result can be used as parameters of `MutableXXX`.

        self.dropout1 = MutableDropout(nni.choice('dropout', [0.25, 0.5, 0.75]))  # choose dropout rate from 0.25, 0.5 and 0.75
        self.dropout2 = nn.Dropout(0.5)
        feature = nni.choice('feature', [64, 128, 256])
        self.fc1 = MutableLinear(9216, feature)
        self.fc2 = MutableLinear(feature, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(self.conv2(x), 2)
        x = torch.flatten(self.dropout1(x), 1)
        x = self.fc2(self.dropout2(F.relu(self.fc1(x))))
        output = F.log_softmax(x, dim=1)
        return output



In [4]:
model_space = MyModelSpace()
model_space

MyModelSpace(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): LayerChoice(
    label='conv2'
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): DepthwiseSeparableConv(
      (depthwise): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32)
      (pointwise): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (dropout1): MutableDropout(p=Categorical([0.25, 0.5, 0.75], label='dropout'))
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): MutableLinear(in_features=9216, out_features=Categorical([64, 128, 256], label='feature'))
  (fc2): MutableLinear(in_features=Categorical([64, 128, 256], label='feature'), out_features=10)
)

Details on model space: https://nni.readthedocs.io/en/stable/nas/construct_space.html

There are basically two exploration approaches:
(1) search by evaluating each sampled model independently, which is the search approach in multi-trial NAS
(2) one-shot weight-sharing based search, which is used in one-shot NAS.
We demonstrate the first approach in this tutorial.

In [5]:
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

def train_epoch(model, device, train_loader, optimizer, epoch):
    loss_fn = torch.nn.CrossEntropyLoss()
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test_epoch(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
          correct, len(test_loader.dataset), accuracy))

    return accuracy

### Pick or customize a model evaluator

In the exploration process, the exploration strategy repeatedly generates new models. A model evaluator is for training and validating each generated model to obtain the model’s performance. The performance is sent to the exploration strategy for the strategy to generate better models.

NNI NAS has provided built-in model evaluators, but to start with, it is recommended to use FunctionalEvaluator, that is, to wrap your own training and evaluation code with one single function. This function should receive one single model class and uses nni.report_final_result() to report the final score of this model.



In [32]:
def evaluate_model(model):
    # By v3.0, the model will be instantiated by default.
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    transf = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    train_loader = DataLoader(MNIST('data/mnist', download=True, transform=transf), batch_size=64, shuffle=True)
    test_loader = DataLoader(MNIST('data/mnist', download=True, train=False, transform=transf), batch_size=64)

    for epoch in range(3):
        # train the model for one epoch
        train_epoch(model, device, train_loader, optimizer, epoch)
        # test the model for one epoch
        accuracy = test_epoch(model, device, test_loader)
        # call report intermediate result. Result can be float or dict
        nni.report_intermediate_result(accuracy)

    # report final test result
    nni.report_final_result(accuracy)

### Create the evaluator

In [33]:
from nni.nas.evaluator import FunctionalEvaluator
evaluator = FunctionalEvaluator(evaluate_model)

### Pick an exploration strategy

See all exploration strategies: https://nni.readthedocs.io/en/stable/nas/exploration_strategy.html

Model evaluators: https://nni.readthedocs.io/en/stable/nas/evaluator.html

In [34]:
import nni.nas.strategy as strategy
search_strategy = strategy.Random()
#search_strategy = strategy.TPE()

### Launch an experiment

In [35]:
from nni.nas.experiment import NasExperiment
exp = NasExperiment(model_space, evaluator, search_strategy)

[2024-04-19 12:46:40] Config is not provided. Will try to infer.
[2024-04-19 12:46:40] Using execution engine based on training service. Trial concurrency is set to 1.
[2024-04-19 12:46:40] Using simplified model format.
[2024-04-19 12:46:40] Using local training service.


In [36]:
# optional

exp.config.max_trial_number = 3   # spawn 3 trials at most
exp.config.trial_concurrency = 1  # will run 1 trial concurrently
#exp.config.trial_gpu_number = 0   # will not use GPU

# uncomment to use GPU
exp.config.trial_gpu_number = 1
exp.config.training_service.use_active_gpu = True

In [37]:
exp.experiment_working_directory = '.'

In [38]:
!nnictl stop

ERROR: There is no experiment running...


In [39]:
exp.run(port = 8896, debug = True)

[2024-04-19 12:46:46] Creating experiment, Experiment ID: hosr3zwx
[2024-04-19 12:46:46] Starting web server...
[2024-04-19 12:46:47] Setting up...
[2024-04-19 12:46:47] Web portal URLs: http://127.0.0.1:8896 http://172.17.0.15:8896
[2024-04-19 12:46:47] Successfully update searchSpace.
[2024-04-19 12:46:47] Checkpoint saved to /root/nni-experiments/hosr3zwx/checkpoint.
[2024-04-19 12:46:47] Experiment initialized successfully. Starting exploration strategy...
[2024-04-19 12:46:48] ERROR: Strategy failed to execute.


ConnectionError: HTTPConnectionPool(host='localhost', port=8896): Max retries exceeded with url: /api/v1/nni/check-status (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f154d070e20>: Failed to establish a new connection: [Errno 111] Connection refused'))

### Export top models

In [17]:
for model_dict in exp.export_top_models(formatter='dict'):
    print(model_dict)

ConnectionError: HTTPConnectionPool(host='localhost', port=8896): Max retries exceeded with url: /api/v1/nni/trial-jobs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f154d0b65b0>: Failed to establish a new connection: [Errno 111] Connection refused'))